In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ib_insync import *
import pandas as pd
from IPython.display import display, clear_output
from datetime import datetime, timedelta
util.startLoop()

In [2]:
ib = IB()
connection = ib.connect()

In [3]:
import keras
model = keras.models.load_model('DNN_model.keras')

In [4]:
model

<Sequential name=sequential_1, built=True>

In [5]:
import pickle
params = pickle.load(open('params.pkl', 'rb'))
mu = params['mu']
std = params['std']

In [6]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)
cfd = CFD('EUR', currency='USD')
ib.qualifyContracts(cfd)
conID=cfd.conId

In [7]:
bars = ib.reqHistoricalData(
    contract,
    endDateTime='',
    durationStr='1 D',
    barSizeSetting='1 min',
    whatToShow='MIDPOINT',
    useRTH=True,
    formatDate=2,
    keepUpToDate=True
)
last_bar_date = bars[-1].date

In [12]:
units = 1000
window = 50
long_sma = 150
momentum_window = 3
lags = 5
threshold = 0.55
start_time = pd.to_datetime(datetime.utcnow())

In [15]:
logger = 'something'

def onBarUpdate(bars, hasNewBar):
    global df, last_bar_date
    global logger
    
    logger += 'run'
    
    if bars[-1].date > last_bar_date:
        logger = 'true, btw units={}'.format(units)
        last_bar_date = bars[-1].date
        
        df = pd.DataFrame(bars)[['date', 'open', 'high', 'low', 'close']]
        df.set_index('date', inplace=True)
        
        ####
        df["returns"] = np.log(df.close / df.close.shift())
        df["dir"] = np.where(df["returns"] > 0, 1, 0)
        df["sma"] = df.close.rolling(window).mean() - df.close.rolling(long_sma).mean()
        df["boll"] = (df.close - df.close.rolling(window).mean()) / df.close.rolling(window).std()
        df["min"] = df.close.rolling(window).min() / df.close - 1
        df["max"] = df.close.rolling(window).max() / df.close - 1
        df["mom"] = df.close.rolling(momentum_window).mean()
        df["vol"] = df.close.rolling(momentum_window).std()
        df['price'] = df['close']
        df.dropna(inplace = True)
        
        # create lags
        cols = []
        features = ["dir", "sma", "boll", "min", "max", "mom", "vol"]

        for f in features:
            for lag in range(1, lags + 1):
                col = "{}_lag_{}".format(f, lag)
                df[col] = df[f].shift(lag)
                cols.append(col)
        df.dropna(inplace = True)
        ####
        # standardization
        df_s = (df - mu) / std
        # predict
        df["probability"] = model.predict(df_s[cols])
        
        #determine positions
        df = df.loc[start_time:].copy() # starting with first live_stream bar (removing historical bars) // but we do it after calculating lag data
        df["position"] = np.where(df.probability < 1- threshold, -1, np.nan)
        df["position"] = np.where(df.probability > threshold, 1, df.position)
        df["position"] = df.position.ffill().fillna(0) # start with neutral position if no strong signal
        #***********************************************************************
        
        
        if df['position'][-1] != 0:
            target = df['position'][-1] * units
            logger += 'gonna execute'
            execute_trade(target = target)

        clear_output(wait=True)
        display(df)
    else:
        logger +=' false'
        pass
        
def execute_trade(target):
    global current_position, logger
    
    logger += 'im here btw target= {} '.format(target) 
    try: 
        logger+= 'trying to execute'
        current_position = [pos.position for pos in ib.positions() if pos.contract.conId == conID][0]
    except:
        logger += 'exception'
        current_position = 0
    
    
    trades = target - current_position
    
    logger += ' | trades = {} | '.format(trades)
    
    if trades > 0:
        logger += 'buy'
        
        side = "BUY"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    elif trades < 0:
        logger += 'sell'
        side = "SELL"
        order = MarketOrder(side, abs(trades))
        trade = ib.placeOrder(cfd, order)
    else:
        logger += '_nobuynosell_'
        pass
        

In [16]:
bars.updateEvent += onBarUpdate

In [21]:
last_bar_date

datetime.datetime(2024, 8, 23, 20, 59, tzinfo=datetime.timezone.utc)